In [1]:
import numpy as np
import pandas as pd

In [143]:
holidays = pd.read_csv('testing_holiday.csv',index_col = 0, parse_dates = ['date'])
train = pd.read_csv('train_prueba.csv',index_col = 0, parse_dates = ['date'])

In [291]:
#Vamos a suponer que day of week de asueto tiene que ser en 4 que es friday
def adjust_holidays(prevdf,daycol,boolcol):
    df = prevdf[prevdf[boolcol] == True]
    lst = []
    for i,dfiter in enumerate(df[daycol]):
        dayweek = dfiter.dayofweek
        dateNum = 4 - dayweek
        newDate = []
        newDate = [pd.Timestamp(dfiter.to_pydatetime() + timedelta(days=dateNum))]
        for item in df[[j for j in df.columns if j not in (daycol,boolcol)]].iloc[i].values:
            newDate.append(item)
        newDate.append(False)
        lst.append(newDate)
    newdf = prevdf[prevdf[boolcol] == False]
    newdf.reset_index(inplace = True, drop = True)
    newreg = pd.DataFrame(lst,columns = df.columns)
    return pd.concat(objs = (newdf,newreg))

holidays_df = pd.read_csv('./data/holidays_events.csv', parse_dates = ['date'])
mask_holidays = holidays_df['date'].dt.year != 2012
holidays_df_new = holidays_df[mask_holidays]
holidays_df_new.reset_index(inplace = True, drop = True)
holidays_df_new = adjust_holidays(holidays_df_new,'date','transferred')

In [288]:
def holidays_on_dataset(df, dfhol, dfcols, dfholcols, dfcolsfinal):
    #values
    dfcols1, dfcols2, dfcols3 = dfcols
    dfholcols1, dfholcols2, dfholval = dfholcols
    
    #local holidays
    local_holidays = pd.merge(left = df[dfcols1], right = dfhol, how = 'inner', 
                             right_on = dfholcols1, left_on = dfcols1)
    local_holidays['is_holiday_local'] = 1
    dfcolsfinal.append('is_holiday_local')
    
    #national holidays
    national_holidays = pd.merge(left = df[[dfcols2]], right = dfhol, how = 'inner', on = dfcols2)
    national_holidays = national_holidays[national_holidays[dfholcols2] == dfholval].reset_index(drop = True)
    national_holidays['is_holiday_national'] = 1
    dfcolsfinal.append('is_holiday_national')
    
    #holidays
    local_national_holidays = pd.concat(objs = (local_holidays, national_holidays)).reset_index(drop = True)
    local_national_holidays = local_national_holidays[dfcols3]
    
    #Fill nulls
    lstvals = [dfholval,0,0]
    for elem,name in enumerate(local_national_holidays.columns[1:]):
        local_national_holidays[name] = local_national_holidays[name].fillna(lstvals[elem])
    
    #new holidays df
    local_national_holidays.drop_duplicates(inplace = True)
    local_national_holidays.sort_values(by = dfcols2, inplace = True)
    local_national_holidays.reset_index(inplace = True, drop = True)
    
    #Join to df
    new_df = pd.merge(left = df, right = local_national_holidays, how = 'left', on = dfcols2)
    
    #grab by columns
    new_df = new_df[dfcolsfinal]
    new_df['is_holiday_local'] = new_df['is_holiday_local'].fillna(0)
    new_df['is_holiday_national'] = new_df['is_holiday_national'].fillna(0)
    
    #Return the df now refactorized
    return new_df


test_df = holidays_on_dataset(train_join_df, holidays_df, 
        [['date','city'],'date',['date','city','is_holiday_local','is_holiday_national']],
        [['date','locale_name'],'locale','National'],
        ['family','date','store_nbr','onpromotion','city_x','state','type','cluster'])
         
test_df

,family,date,store_nbr,onpromotion,city_x,state,type,cluster,is_holiday_local,is_holiday_national
0,AUTOMOTIVE,2013-01-01,1,0,Quito,Pichincha,D,13,0.0,1.0
1,AUTOMOTIVE,2013-01-02,1,0,Quito,Pichincha,D,13,0.0,0.0
2,AUTOMOTIVE,2013-01-03,1,0,Quito,Pichincha,D,13,0.0,0.0
3,AUTOMOTIVE,2013-01-04,1,0,Quito,Pichincha,D,13,0.0,0.0
4,AUTOMOTIVE,2013-01-05,1,0,Quito,Pichincha,D,13,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
3038305,SEAFOOD,2017-08-11,54,0,El Carmen,Manabi,C,3,0.0,1.0
3038306,SEAFOOD,2017-08-12,54,1,El Carmen,Manabi,C,3,0.0,0.0
3038307,SEAFOOD,2017-08-13,54,0,El Carmen,Manabi,C,3,0.0,0.0
3038308,SEAFOOD,2017-08-14,54,0,El Carmen,Manabi,C,3,0.0,0.0


In [240]:
stores_df = pd.read_csv('./data/stores.csv')
train_df = pd.read_csv('./data/train.csv', parse_dates = ['date'], index_col = 0)
train_group_df = train_df.groupby(by=['family','date','store_nbr']).sum().reset_index()
train_join_df = pd.merge(left = train_group_df, right = stores_df, on = 'store_nbr', how = 'inner')

In [333]:
train_join_df

,family,date,store_nbr,sales,onpromotion,city,state,type,cluster
0,AUTOMOTIVE,2013-01-01,1,0.0,0,Quito,Pichincha,D,13
1,AUTOMOTIVE,2013-01-02,1,2.0,0,Quito,Pichincha,D,13
2,AUTOMOTIVE,2013-01-03,1,3.0,0,Quito,Pichincha,D,13
3,AUTOMOTIVE,2013-01-04,1,3.0,0,Quito,Pichincha,D,13
4,AUTOMOTIVE,2013-01-05,1,5.0,0,Quito,Pichincha,D,13
...,...,...,...,...,...,...,...,...,...
3000883,SEAFOOD,2017-08-11,54,0.0,0,El Carmen,Manabi,C,3
3000884,SEAFOOD,2017-08-12,54,1.0,1,El Carmen,Manabi,C,3
3000885,SEAFOOD,2017-08-13,54,2.0,0,El Carmen,Manabi,C,3
3000886,SEAFOOD,2017-08-14,54,0.0,0,El Carmen,Manabi,C,3


In [336]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer

def scaling(type_scaler,method,df,cols):
    if type_scaler == 'Standard':
        scaler = StandardScaler()
    
    elif type_sclaer == 'Power':
        scaler = PowerTransformer(method = method)
    
    col1 = cols[0] + '_scale'
    col2 = cols[1] + '_scale'
    
    df[col1], df[col2] = scaler.fit_transform(df[cols])
    print(df)
    return scaler

def invserse_scaling(scaler,df,cols):
    for i in cols:
        df[i + 'original_value'] = scaler.inverse_transform(df[i])

In [337]:
#standard scaling
standard = scaling('Standard','',train_dataset_clean1,['sales','oil_price'])
#train_dataset_clean1.sort_values(by='sales')
#inverse_scaling(standard,train_dataset_clean1,['sales','oil_price'])
#train_dataset_clean1

[[-0.32424664  0.98664192]
 [-0.32243784  0.98664192]
 [-0.32153345  0.98664192]
 ...
 [-0.32243784 -0.73314599]
 [-0.32424664 -0.73314599]
 [-0.32153345 -0.74704042]]


In [312]:
def scaling2(type_scaler,method,df,cols):
    data = df[cols]
    
    if type_scaler == 'Standard':
        ct = ColumnTransformer([
            ('somename',StandardScaler(),cols)], remainder = 'passthrough')
    
    elif type_sclaer == 'Power':
        ct = ColumnTransformer([
            ('somename',PowerTransformer(method = method),cols)], remainder = 'passthrough')
    
    return ct.fit_transform(data)

In [313]:
standard2 = scaling2('Standard','',train_dataset_clean1,['sales','oil_price'])
standard2

array([[-0.32424664,  0.98664192],
       [-0.32243784,  0.98664192],
       [-0.32153345,  0.98664192],
       ...,
       [-0.32243784, -0.73314599],
       [-0.32424664, -0.73314599],
       [-0.32153345, -0.74704042]])

In [305]:
train_dataset_clean1 = pd.read_csv('./data/train_join_df_encoded.csv')
train_dataset_clean1.drop('date', axis=1, inplace=True)

In [335]:
train_dataset_clean2 = pd.read_csv('./data/train_join_df_encoded.csv')
train_dataset_clean2.drop('date', axis=1, inplace=True)
train_dataset_clean2.sort_values(by='sales')

,store_nbr,sales,onpromotion,cluster,Year,Month,Week,Day,Dayofweek,Dayofyear,is_holiday_local,is_holiday_national,Is_quarter_end_encode,Is_quarter_start_encode,family_encode,city_x_encode,state_encode,type_encode,oil_price
0,1,0.000,0,13,2013,1,1,1,1,1,0.0,1.0,0,1,0,18,12,3,93.140
1713589,31,0.000,0,10,2013,3,10,6,2,65,0.0,0.0,0,0,15,1,9,1,91.318
1713588,31,0.000,0,10,2013,3,10,5,1,64,0.0,0.0,0,0,15,1,9,1,91.668
1713587,31,0.000,0,10,2013,3,10,4,0,63,0.0,0.0,0,0,15,1,9,1,92.190
1713586,31,0.000,0,10,2013,3,9,3,6,62,0.0,0.0,0,0,15,1,9,1,92.190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2497331,45,76090.000,38,11,2016,4,16,18,0,109,0.0,1.0,0,0,12,18,12,0,41.226
1090710,20,87438.516,53,6,2016,4,16,21,3,112,0.0,1.0,0,0,12,18,12,1,41.384
1090709,20,87438.516,53,6,2016,4,16,21,3,112,1.0,0.0,0,0,12,18,12,1,41.038
2180377,39,89576.360,0,6,2016,10,40,7,4,281,1.0,0.0,0,0,24,3,0,1,49.076


In [ ]:
#Power transforming yeo-johnson
yeo_johnson = scaling('Power','yeo-johnson',train_dataset_clean2,'sales','oil_price')
inverse_scaling(yeo_johnson,train_dataset_clean2,'sales','oil_price')
train_dataset_clean2

In [ ]:
train_dataset_clean3 = pd.read_csv('./data/train_join_df_encoded.csv')
train_dataset_clean3.drop('date', axis=1, inplace=True)

In [ ]:
#Power transforming box-cox
box_cox = scaling('Power','box-cox',train_dataset_clean3,'sales','oil_price')
inverse_scaling(box_cox,train_dataset_clean3,'sales','oil_price')
train_dataset_clean3